## Setup

In [1]:
import sys
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
sys.path.append('../SignalBacktesting/') # god this is awful
from backtesting import backtesting
from betterMT5 import connected
from datetime import datetime
import cufflinks as cf
import pandas as pd

In [2]:
with connected():
    test = backtesting.Backtest("../SignalBacktesting/chats/results_daniel.json", verbose=None)

In [3]:
with connected():
    results = test.run()

In [15]:
def gen_results(test_run = test.run_results, ignore = None):
    results = pd.DataFrame(test.make_results(test_run, ignore=ignore))
    results.drop(results[results.result>4].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips>50].index, axis=0, inplace=True)
    results.drop(results[results.sl_pips<5].index, axis=0, inplace=True)
    results.open = pd.to_datetime(results.open.apply(str), utc=True)
    results.close = pd.to_datetime(results.close.apply(str), utc=True)
    return results

In [84]:
def gen_multi(results: list, names: list, attr: str = 'result'):
    to_pass = {k: v.set_index('open') for k, v in zip(names, results)}
    return pd.concat(to_pass, axis=0).reset_index()

def get_attr_multi(multi, attr):
    return multi.pivot('open', 'level_0', attr)

## Results (Vanilla Daniel Savage)

In [12]:
results = gen_results()
data_loss = len(test.trades) - results.shape[0]
print(f'Post-adjustment data loss: {data_loss} ({data_loss/len(test.trades):.2%})')

Post-adjustment data loss: 21 (4.90%)


In [77]:
results.iplot(kind='scatter', mode='markers', x='close', title='Results (R) by type', yTitle='R',
              y='result', categories='type', size=6, colors='red blue purple green grey black'.split(' '))

In [85]:
results_ignore_close = gen_results(ignore=['CLOSE'])
results_ignore_sl_to_be = gen_results(ignore=['SL_TO_BE'])
results_ignore_partials = gen_results(ignore=['PARTIALS'])
multi = gen_multi(results=[results, results_ignore_close, results_ignore_sl_to_be, results_ignore_partials],
                  names='normal ignore_close ignore_be ignore_partials'.split(' '))


In [90]:
get_attr_multi(multi, 'sl_pips').iplot(kind='hist', barmode='group', title='SL (pips) distribution', yTitle='pips', 
    xTitle='index', opacity=1, colors='blue red purple black'.split(' '), linecolor='grey')

In [109]:
df = get_attr_multi(multi, 'close')
df = df.apply(lambda x: (x-df.index).dt.total_seconds()/60).reset_index().drop('open', axis=1)
df.iplot(kind='hist', barmode='group', title='Trade length distribution (minutes)', yTitle='minutes', 
    xTitle='index', opacity=1, colors='blue red purple black'.split(' '), linecolor='grey')
del df

In [120]:
interpolated = get_attr_multi(multi, 'result').cumsum().resample('3600T').mean().interpolate(method='spline', order=3)
interpolated.iplot(kind='scatter', fill=False, title='Trade results by type', yTitle='R', 
    xTitle='date', opacity=1, zerolinecolor='grey', colors='blue red purple black'.split(' '))
del interpolated

In [70]:
res_gb_M = results.set_index('close').groupby(pd.Grouper(freq='M')).sum()[['result']]
res_gb_M['result_pos'] = res_gb_M.result[res_gb_M.result>=0]
res_gb_M['result_neg'] = res_gb_M.result[res_gb_M.result<0]
res_gb_M.drop('result', inplace=True, axis=1)
res_gb_M

,result_pos,result_neg
close,,
2020-11-30 00:00:00+00:00,NaN,-2.973839
2020-12-31 00:00:00+00:00,NaN,-1.111851
2021-01-31 00:00:00+00:00,1.130078,NaN
2021-02-28 00:00:00+00:00,NaN,-1.119601
2021-03-31 00:00:00+00:00,3.874042,NaN
2021-04-30 00:00:00+00:00,NaN,-0.163038
2021-05-31 00:00:00+00:00,3.559188,NaN
2021-06-30 00:00:00+00:00,NaN,-1.596187
2021-07-31 00:00:00+00:00,1.800060,NaN


In [71]:
avg = res_gb_M.result_pos.mean() + res_gb_M.result_neg.mean()
res_gb_M.iplot(kind='bar', bargap=0, barmode='stack', colors=dict(result_pos='seagreen', result_neg='#FF0000'), hline=avg)

In [77]:

res_gb_D = results.set_index('open')[['result']].groupby(pd.Grouper(freq='D')).sum()
res_gb_D['result_pos'] = res_gb_D.result[res_gb_D.result>=0]
res_gb_D['result_neg'] = res_gb_D.result[res_gb_D.result<0]
res_gb_D.drop('result', inplace=True, axis=1)
res_gb_D

,result_pos,result_neg
open,,
2020-11-05 00:00:00+00:00,NaN,-2.000000
2020-11-06 00:00:00+00:00,NaN,-0.032787
2020-11-07 00:00:00+00:00,0.000000,NaN
2020-11-08 00:00:00+00:00,0.000000,NaN
2020-11-09 00:00:00+00:00,NaN,-0.036290
...,...,...
2022-01-28 00:00:00+00:00,0.000000,NaN
2022-01-29 00:00:00+00:00,0.000000,NaN
2022-01-30 00:00:00+00:00,0.000000,NaN


In [78]:
avg = res_gb_D.result_pos.mean() + res_gb_D.result_neg.mean()
res_gb_D.iplot(kind='scatter', mode='markers', colors=dict(result_pos='seagreen', result_neg='#FF0000'), hline=avg)

In [80]:
results.set_index('open').loc['2021-12-7']

,close,symbol,sl_pips,result,type,cum,duration_minutes
open,,,,,,,
2021-12-07 08:12:08+00:00,2021-12-07 08:33:00+00:00,GBPJPY,17.9,-1.0,SL,1.104287,20.866667
2021-12-07 08:52:00+00:00,2021-12-07 09:09:00+00:00,GBPJPY,19.8,-1.0,SL,0.104287,17.000000
2021-12-07 09:17:55+00:00,2021-12-07 10:52:00+00:00,GBPJPY,20.7,-1.0,SL,-0.895713,94.083333
2021-12-07 11:01:07+00:00,2021-12-07 11:23:00+00:00,GBPJPY,13.9,-1.0,SL,-1.895713,21.883333
2021-12-07 12:23:18+00:00,2021-12-07 13:18:00+00:00,GBPJPY,23.0,-1.0,SL,-2.895713,54.700000
